In [22]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
max_profit = 0
min_loss = 0
risk = 0 # number of stocks currently in position

In [23]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [24]:
symbols = """
select distinct(name) from stockdatadaily order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [25]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [26]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = 0.0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatadaily order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [27]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatadaily where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
10 % done
20 % done
30 % done
40 % done
50 % done
60 % done
70 % done
80 % done
90 % done


In [28]:
sql_dates = """
select distinct(date) from stockdatadaily order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Loading KNN model </h1>

In [29]:
import pickle
# model = tf.keras.models.load_model('Best1.model')
pickle_in = open("KNN-DailyModel63_46days.pickle","rb")
model = pickle.load(pickle_in)

<h1> Trading strategy with the training sets generation

In [30]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global risk
    global max_profit
    global min_loss
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatadaily where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        training_inst = training_set_extractor(data_till_date, date)
                        training_inst = np.asarray(training_inst)
                        for_predict = np.append([tf.keras.utils.normalize(training_inst[0])], [tf.keras.utils.normalize(training_inst[1])])
                        if(model.predict([for_predict]) == 1):
                            buy_stocks(current_price, current_symbol, date)                         
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                
                                win += 1
                                current_symbol.flag_for_wins = True
                                risk = risk - 1
                        
                        
                    if current_symbol.stop_order > current_price:
                        risk = risk - 1                        
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            
                            if current_symbol.flag_for_wins == False:
                                win += 1
                                
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1
                            
                            current_symbol.flag_for_wins = False
                    
                            
                        profit += profit_today
                        
                        if profit > max_profit:
                            max_profit = profit
                        if profit < min_loss:
                            min_loss = profit
                        
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [31]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global risk
    global commission
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
        risk += 1 
    else:
        pass

def training_set_extractor(data, date):
    lastFrom = pd.to_datetime(date)

    destDate = lastFrom - pd.Timedelta(days=45)
    lastFrom = str(lastFrom.date())
    destDate = str(destDate.date())

    df = pd.DataFrame()
    df = data[(data['date'] >= destDate) & (data['date'] <= lastFrom)]

    df = df.set_index('date')

    df.index = pd.DatetimeIndex(df.index)
    idx = pd.date_range(destDate, lastFrom)
    df = df.reindex(idx, fill_value=0)
    
    highs = df['2. high'].tolist()
    volumes = df['5. volume'].tolist()
    lows = df['3. low'].tolist()
    
    for i in range(1, len(highs)):
        if highs[i] == 0:
            highs[i] = highs[i-1]
        if volumes[i] == 0:
            volumes[i] = volumes[i-1]
        if lows[i] == 0:
            lows[i] = lows[i-1]
        
    
    training_instance = [highs, volumes, lows, 'na']
    
    return training_instance[:] 
   

In [32]:
automatic_trader(symbol_list)

Buying  42.0 stocks of  KOF At price 19.25 on date 2000-02-10
Buying  317.0 stocks of  MUE At price 13.49 on date 2001-11-13
Selling  317.0 stocks of MUE  at price 12.82 on date 2001-11-27
Buying  717.0 stocks of  JMBA At price 7.03 on date 2005-08-09
Buying  175.0 stocks of  CSBR At price 0.7 on date 2007-06-06
Buying  44.0 stocks of  CLNE At price 13.679 on date 2007-06-18
Selling  717.0 stocks of JMBA  at price 5.76 on date 2007-10-29
Selling  44.0 stocks of CLNE  at price 10.99 on date 2008-10-23
Buying  26.0 stocks of  EOT At price 22.7091 on date 2012-08-30
Buying  26.0 stocks of  MUE At price 16.05 on date 2012-12-06
Selling  42.0 stocks of KOF  at price 143.26 on date 2013-07-05
Buying  17.0 stocks of  JMBA At price 14.8 on date 2013-07-31
Buying  229.0 stocks of  BMCH At price 19.62 on date 2016-05-27
Buying  102.0 stocks of  USFD At price 25.435 on date 2016-06-09
Selling  229.0 stocks of BMCH  at price 15.85 on date 2016-11-03
Buying  217.0 stocks of  BOTZ At price 21.28 on 

In [34]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  7692.750800000001
Minimum Loss is  -1256.296
Profit 7692.750800000001
Remaining cash  26231.7174
Win Percentage
41.66666666666667
